In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#modeling
import os
import sys
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import warnings


In [2]:
df = pd.read_csv(r"data\dataset.csv")
df.head()


,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Timestamp,Account_Balance,Device_Type,Location,Merchant_Category,IP_Address_Flag,...,Daily_Transaction_Count,Avg_Transaction_Amount_7d,Failed_Transaction_Count_7d,Card_Type,Card_Age,Transaction_Distance,Authentication_Method,Risk_Score,Is_Weekend,Fraud_Label
0,TXN_33553,USER_1834,39.79,POS,2023-08-14 19:30:00,93213.17,Laptop,Sydney,Travel,0,...,7,437.63,3,Amex,65,883.17,Biometric,0.8494,0,0
1,TXN_9427,USER_7875,1.19,Bank Transfer,2023-06-07 04:01:00,75725.25,Mobile,New York,Clothing,0,...,13,478.76,4,Mastercard,186,2203.36,Password,0.0959,0,1
2,TXN_199,USER_2734,28.96,Online,2023-06-20 15:25:00,1588.96,Tablet,Mumbai,Restaurants,0,...,14,50.01,4,Visa,226,1909.29,Biometric,0.8400,0,1
3,TXN_12447,USER_2617,254.32,ATM Withdrawal,2023-12-07 00:31:00,76807.20,Tablet,New York,Clothing,0,...,8,182.48,4,Visa,76,1311.86,OTP,0.7935,0,1
4,TXN_39489,USER_2014,31.28,POS,2023-11-11 23:44:00,92354.66,Mobile,Mumbai,Electronics,0,...,14,328.69,4,Mastercard,140,966.98,Password,0.3819,1,1


In [3]:
df.tail()

,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Timestamp,Account_Balance,Device_Type,Location,Merchant_Category,IP_Address_Flag,...,Daily_Transaction_Count,Avg_Transaction_Amount_7d,Failed_Transaction_Count_7d,Card_Type,Card_Age,Transaction_Distance,Authentication_Method,Risk_Score,Is_Weekend,Fraud_Label
49995,TXN_11284,USER_4796,45.05,Online,2023-01-29 18:38:00,76960.11,Mobile,Tokyo,Clothing,0,...,2,389.00,3,Amex,98,1537.54,PIN,0.1493,1,0
49996,TXN_44732,USER_1171,126.15,POS,2023-05-09 08:55:00,28791.75,Mobile,Tokyo,Clothing,0,...,13,434.95,4,Visa,93,2555.72,Biometric,0.3653,0,1
49997,TXN_38158,USER_2510,72.02,Online,2023-01-30 19:32:00,29916.41,Laptop,Mumbai,Clothing,0,...,1,369.15,2,Visa,114,4686.59,Biometric,0.5195,0,0
49998,TXN_860,USER_2248,64.89,Bank Transfer,2023-03-09 19:47:00,67895.67,Mobile,Tokyo,Electronics,0,...,13,242.29,4,Discover,72,4886.92,Biometric,0.7063,0,1
49999,TXN_15795,USER_6529,13.00,Bank Transfer,2023-08-19 23:57:00,7668.82,Tablet,London,Restaurants,0,...,5,273.78,1,Mastercard,154,1568.95,OTP,0.8938,0,1


In [4]:
df.dtypes

Transaction_ID                   object
User_ID                          object
Transaction_Amount              float64
Transaction_Type                 object
Timestamp                        object
Account_Balance                 float64
Device_Type                      object
Location                         object
Merchant_Category                object
IP_Address_Flag                   int64
Previous_Fraudulent_Activity      int64
Daily_Transaction_Count           int64
Avg_Transaction_Amount_7d       float64
Failed_Transaction_Count_7d       int64
Card_Type                        object
Card_Age                          int64
Transaction_Distance            float64
Authentication_Method            object
Risk_Score                      float64
Is_Weekend                        int64
Fraud_Label                       int64
dtype: object

In [5]:
df['Transaction_Type'].unique()

array(['POS', 'Bank Transfer', 'Online', 'ATM Withdrawal'], dtype=object)

In [6]:

df.drop(columns=['Risk_Score'],inplace=True)

In [7]:
df.nunique()

Transaction_ID                  50000
User_ID                          8963
Transaction_Amount              21763
Transaction_Type                    4
Timestamp                       47724
Account_Balance                 49867
Device_Type                         3
Location                            5
Merchant_Category                   5
IP_Address_Flag                     2
Previous_Fraudulent_Activity        2
Daily_Transaction_Count            14
Avg_Transaction_Amount_7d       31420
Failed_Transaction_Count_7d         5
Card_Type                           4
Card_Age                          239
Transaction_Distance            47546
Authentication_Method               4
Is_Weekend                          2
Fraud_Label                         2
dtype: int64

In [8]:

df['Timestamp'] = pd.to_datetime(df['Timestamp'])


In [9]:
df.head()

,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Timestamp,Account_Balance,Device_Type,Location,Merchant_Category,IP_Address_Flag,Previous_Fraudulent_Activity,Daily_Transaction_Count,Avg_Transaction_Amount_7d,Failed_Transaction_Count_7d,Card_Type,Card_Age,Transaction_Distance,Authentication_Method,Is_Weekend,Fraud_Label
0,TXN_33553,USER_1834,39.79,POS,2023-08-14 19:30:00,93213.17,Laptop,Sydney,Travel,0,0,7,437.63,3,Amex,65,883.17,Biometric,0,0
1,TXN_9427,USER_7875,1.19,Bank Transfer,2023-06-07 04:01:00,75725.25,Mobile,New York,Clothing,0,0,13,478.76,4,Mastercard,186,2203.36,Password,0,1
2,TXN_199,USER_2734,28.96,Online,2023-06-20 15:25:00,1588.96,Tablet,Mumbai,Restaurants,0,0,14,50.01,4,Visa,226,1909.29,Biometric,0,1
3,TXN_12447,USER_2617,254.32,ATM Withdrawal,2023-12-07 00:31:00,76807.20,Tablet,New York,Clothing,0,0,8,182.48,4,Visa,76,1311.86,OTP,0,1
4,TXN_39489,USER_2014,31.28,POS,2023-11-11 23:44:00,92354.66,Mobile,Mumbai,Electronics,0,1,14,328.69,4,Mastercard,140,966.98,Password,1,1


In [10]:
df = df.sort_values(by=['User_ID','Timestamp']).reset_index(drop=True)

# tx_count_last_24h avg_amount_7d failed_tx_count_7d has_prior_fraud is_new_user

In [11]:
df_temp = df.set_index('Timestamp')

In [12]:
df.drop(columns=['Transaction_ID'],axis=1,inplace=True)
leakage_columns = [
                'Previous_Fraudulent_Activity',  # Future information
                'Failed_Transaction_Count_7d',   # Includes current transaction
                'IP_Address_Flag',              # Calculated using fraud patterns                    # Calculated using fraud labels
            ]
df = df.drop(columns=leakage_columns)

In [13]:
df['hour'] = pd.to_datetime(df['Timestamp']).dt.hour
df['day_of_week'] = pd.to_datetime(df['Timestamp']).dt.dayofweek
df['day_of_month'] = pd.to_datetime(df['Timestamp']).dt.month


In [14]:
df.nunique()

User_ID                       8963
Transaction_Amount           21763
Transaction_Type                 4
Timestamp                    47724
Account_Balance              49867
Device_Type                      3
Location                         5
Merchant_Category                5
Daily_Transaction_Count         14
Avg_Transaction_Amount_7d    31420
Card_Type                        4
Card_Age                       239
Transaction_Distance         47546
Authentication_Method            4
Is_Weekend                       2
Fraud_Label                      2
hour                            24
day_of_week                      7
day_of_month                    12
dtype: int64

In [15]:
x = df.drop('Fraud_Label',axis=1)
y = df['Fraud_Label']

In [16]:
x.columns


Index(['User_ID', 'Transaction_Amount', 'Transaction_Type', 'Timestamp',
       'Account_Balance', 'Device_Type', 'Location', 'Merchant_Category',
       'Daily_Transaction_Count', 'Avg_Transaction_Amount_7d', 'Card_Type',
       'Card_Age', 'Transaction_Distance', 'Authentication_Method',
       'Is_Weekend', 'hour', 'day_of_week', 'day_of_month'],
      dtype='object')

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [18]:
def get(df):
    user_stats = {}
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['user_transaction_count'] = 0
    df['user_avg_amount'] = 0
    df['user_amount_std'] = 0
    df['days_since_first_transaction'] = 0

    for i, row in df.iterrows():
        user = row['User_ID']
        amount = row['Transaction_Amount']
        t = row['Timestamp']
        
        if user not in user_stats:
            # first ever transaction
            user_stats[user] = {
                "amounts": [amount],
                "first_ts": t
            }
            
            df.at[i, 'user_transaction_count'] = 1
            df.at[i, 'user_avg_amount'] = amount
            df.at[i, 'user_amount_std'] = 0
            df.at[i, 'days_since_first_transaction'] = 0
        
        else:
            history = user_stats[user]["amounts"]
            first_ts = user_stats[user]["first_ts"]
            
            df.at[i, 'user_transaction_count'] = len(history) + 1
            df.at[i, 'user_avg_amount'] = np.mean(history)
            df.at[i, 'user_amount_std'] = np.std(history)
            df.at[i, 'days_since_first_transaction'] = (t - first_ts).total_seconds() / 86400
            
            # update history AFTER using it
            history.append(amount)
    df = df.drop(columns=['User_ID','Timestamp'])
    return df
x_train = get(x_train)

C:\Users\onkar\AppData\Local\Temp\ipykernel_23504\1870627780.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '179.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'user_avg_amount'] = amount
C:\Users\onkar\AppData\Local\Temp\ipykernel_23504\1870627780.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '166.4388888888889' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'days_since_first_transaction'] = (t - first_ts).total_seconds() / 86400
C:\Users\onkar\AppData\Local\Temp\ipykernel_23504\1870627780.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '76.15' has dtype incompatible with int64, please explicitly cast to a compatible dtype

In [19]:
pd.DataFrame(x_train).columns

Index(['Transaction_Amount', 'Transaction_Type', 'Account_Balance',
       'Device_Type', 'Location', 'Merchant_Category',
       'Daily_Transaction_Count', 'Avg_Transaction_Amount_7d', 'Card_Type',
       'Card_Age', 'Transaction_Distance', 'Authentication_Method',
       'Is_Weekend', 'hour', 'day_of_week', 'day_of_month',
       'user_transaction_count', 'user_avg_amount', 'user_amount_std',
       'days_since_first_transaction'],
      dtype='object')

In [20]:
one_hot_enc = list()
num = list()
x = pd.DataFrame(x_train)
for i in x.columns:
    if x[i].nunique() <= 15:
        one_hot_enc.append(i)
    else:
        num.append(i)

one_hot_enc


['Transaction_Type',
 'Device_Type',
 'Location',
 'Merchant_Category',
 'Daily_Transaction_Count',
 'Card_Type',
 'Authentication_Method',
 'Is_Weekend',
 'day_of_week',
 'day_of_month',
 'user_transaction_count']

In [21]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
onehot = OneHotEncoder()

processor = ColumnTransformer(
    [
        ('One hot encoder',onehot,one_hot_enc),
        ('Standard Scaler',scaler,num)
    ]
)

In [22]:
x_train = processor.fit_transform(x_train)


In [23]:
from sklearn.metrics import mean_squared_error,r2_score

In [24]:
def evaluate_model(true,pred):
    mae = mean_squared_error(true,pred)
    score = r2_score(true,pred)
    return mae,score

In [25]:
df.isnull().sum()

User_ID                      0
Transaction_Amount           0
Transaction_Type             0
Timestamp                    0
Account_Balance              0
Device_Type                  0
Location                     0
Merchant_Category            0
Daily_Transaction_Count      0
Avg_Transaction_Amount_7d    0
Card_Type                    0
Card_Age                     0
Transaction_Distance         0
Authentication_Method        0
Is_Weekend                   0
Fraud_Label                  0
hour                         0
day_of_week                  0
day_of_month                 0
dtype: int64

In [26]:
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "KNN": KNeighborsClassifier(),
    "XGBoost": XGBClassifier(random_state=42, eval_metric='logloss'),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000)
}

In [27]:
def build_user_features(row):
    user = row['User_ID']
    amount = row['Transaction_Amount']
    t = pd.to_datetime(row['Timestamp'])

    if user in user_state:
        stats = user_state[user]

        count = stats["count"]
        avg = stats["avg"]
        std = stats["std"]
        first_ts = stats["first_ts"]

        days_since_first = (t - first_ts).total_seconds() / 86400

    else:
        # defaults for brand-new users
        count = 1
        avg = amount
        std = 0
        days_since_first = 0

    deviation = (amount - avg) / (std + 1e-5)

    return pd.Series([
        count, avg, std, deviation, days_since_first
    ])


In [28]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

forest = RandomForestClassifier(
    n_estimators=300,
    max_depth=6,
    class_weight='balanced',
    min_samples_split=4
)

forest.fit(x_train, y_train)

y_pred_test = forest.predict(x_test)

# Confusion matrix
conf = confusion_matrix(y_test, y_pred_test)
print("Confusion Matrix:\n", conf)

# Accuracy
acc = accuracy_score(y_test, y_pred_test)
print("\nAccuracy:", acc)

# Precision, Recall, F1 for each class
report = classification_report(y_test, y_pred_test)
print("\nClassification Report:\n", report)


c:\Users\onkar\OneDrive\Fraud Detection\venv\lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'USER_7037'

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    roc_auc_score
)
fraud_ratio = (y_train == 0).sum() / (y_train == 1).sum()
# Train model
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    scale_pos_weight=fraud_ratio,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb.fit(x_train, y_train)

# Predictions
y_pred = xgb.predict(x_test)

# Confusion Matrix
conf = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print("\nAccuracy:", acc)

# Precision / Recall / F1
report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", report)

# AUC (binary only)
y_prob = xgb.predict_proba(x_test)[:, 1]
auc = roc_auc_score(y_test, y_prob)
print("\nROC AUC:", auc)


Confusion Matrix:
 [[4517 2329]
 [ 961 2193]]

Accuracy: 0.671

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.66      0.73      6846
           1       0.48      0.70      0.57      3154

    accuracy                           0.67     10000
   macro avg       0.65      0.68      0.65     10000
weighted avg       0.72      0.67      0.68     10000


ROC AUC: 0.7494998213250622


In [ ]:
df.nunique()

Transaction_Amount                21763
Transaction_Type                      4
Account_Balance                   49867
Device_Type                           3
Location                              5
Merchant_Category                     5
Daily_Transaction_Count              14
Avg_Transaction_Amount_7d         31420
Card_Type                             4
Card_Age                            239
Transaction_Distance              47546
Authentication_Method                 4
Is_Weekend                            2
Fraud_Label                           2
user_transaction_count               16
user_fraud_rate                      52
user_avg_amount                    8821
user_amount_std                    8766
amount_deviation_from_user_avg    49792
days_since_first_transaction      39414
hour                                 24
day_of_week                           7
day_of_month                         12
dtype: int64

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    roc_auc_score
)

# Model
ada = AdaBoostClassifier(
    n_estimators=300,
    learning_rate=0.05,
    random_state=42
)

# Train
ada.fit(x_train, y_train)

# Predict
y_pred = ada.predict(x_test)

# Confusion Matrix
conf = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print("\nAccuracy:", acc)

# Precision / Recall / F1
report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", report)

# AUC (binary only)
y_prob = ada.predict_proba(x_test)[:, 1]
auc = roc_auc_score(y_test, y_prob)
print("\nROC AUC:", auc)


ValueError: Input X contains NaN.
AdaBoostClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    roc_auc_score
)

# Model
gb = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

# Train
gb.fit(x_train, y_train)

# Predict
y_pred = gb.predict(x_test)

# Confusion Matrix
conf = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print("\nAccuracy:", acc)

# Precision / Recall / F1
report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", report)

# AUC (binary only)
y_prob = gb.predict_proba(x_test)[:, 1]
auc = roc_auc_score(y_test, y_prob)
print("\nROC AUC:", auc)


Confusion Matrix:
 [[6826   20]
 [1150 2004]]

Accuracy: 0.883

Classification Report:
               precision    recall  f1-score   support

           0       0.86      1.00      0.92      6846
           1       0.99      0.64      0.77      3154

    accuracy                           0.88     10000
   macro avg       0.92      0.82      0.85     10000
weighted avg       0.90      0.88      0.87     10000


ROC AUC: 0.9100651186321929


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras import Sequential
from tensorflow.keras.losses import MeanSquaredError,BinaryFocalCrossentropy
from tensorflow.keras.activations import sigmoid

model = Sequential(
    [
        Input(shape=(x_train.shape[1],)),
        Dense(units=128,activation='relu'),
        Dense(units=64,activation='relu'),
        Dense(units=32,activation='relu'),
        Dense(units=1,activation='sigmoid')
    ],name='my_model'
)

In [ ]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │        10,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,138 (74.76 KB)

 Trainable params: 19,138 (74.76 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
[l1,l2,l3,l4] = model.layers
model.compile(
    optimizer='adam',
    loss= BinaryFocalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
x_train.shape

(40000, 83)

In [ ]:
y_train.shape

(40000,)

In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8713 - loss: 0.0840
Epoch 2/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8810 - loss: 0.0777
Epoch 3/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.8811 - loss: 0.0768
Epoch 4/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.8814 - loss: 0.0760
Epoch 5/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.8829 - loss: 0.0747
Epoch 6/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8839 - loss: 0.0729
Epoch 7/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8865 - loss: 0.0706
Epoch 8/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8901 - loss: 0.0679
Epoch 9/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8928 - loss: 0.0646
Epoch 10/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8989 - loss: 0.0613
Epoch 11/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9043 - loss: 0.0572
Epoch 12/50
1250/1250 ━━━━━━━

In [ ]:
y_prob = model.predict(x_test)   # probabilities (0–1)

y_pred = (y_prob > 0.5).astype("int32")


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    roc_auc_score
)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
print("\nAccuracy:", accuracy_score(y_test, y_pred))

print("\nClassification Report:\n",
      classification_report(y_test, y_pred))
auc = roc_auc_score(y_test, y_prob)
print("\nROC AUC:", auc)


Confusion Matrix:
 [[5994  852]
 [ 904 2250]]

Accuracy: 0.8244

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.88      0.87      6846
           1       0.73      0.71      0.72      3154

    accuracy                           0.82     10000
   macro avg       0.80      0.79      0.80     10000
weighted avg       0.82      0.82      0.82     10000


ROC AUC: 0.8768104152390733


In [ ]:
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
print(neg_count)
print(pos_count)
total = len(y_train)
pos_count/neg_count

27087
12913


np.float64(0.47672315132720494)

In [ ]:
class_weight = {
    0: (1 / neg_count) * (total / 2.0),
    1: (1 / pos_count) * (total / 2.0)
}

print(f"Class weights: {class_weight}")

Class weights: {0: np.float64(0.7383615756636024), 1: np.float64(1.548826763726477)}


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9898 - loss: 0.0074 - val_accuracy: 0.9821 - val_loss: 0.0140
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization

# Calculate exact class weights from your data
neg_count = 27087  # Class 0
pos_count = 12913  # Class 1
total = neg_count + pos_count

class_weight = {
    0: total / (2 * neg_count),  # = 0.738
    1: total / (2 * pos_count)   # = 1.549
}

print(f"Class weights: {class_weight}")
# Output: {0: 0.738, 1: 1.549}

# Build model
model = Sequential([
    Input(shape=(x_train.shape[1],)),
    
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(32, activation='relu'),
    Dropout(0.2),
    
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Recall(name='recall')]
)

# Train with class weights
history = model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=256,
    class_weight=class_weight,  # ← KEY!
    verbose=1
)

Class weights: {0: 0.7383615756636025, 1: 1.5488267637264772}
Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.7641 - loss: 0.4942 - recall: 0.7223 - val_accuracy: 0.8761 - val_loss: 0.3859 - val_recall: 0.6398
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8529 - loss: 0.3701 - recall: 0.7093 - val_accuracy: 0.8767 - val_loss: 0.3547 - val_recall: 0.6511
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8639 - loss: 0.3561 - recall: 0.7029 - val_accuracy: 0.8746 - val_loss: 0.3385 - val_recall: 0.6759
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8687 - loss: 0.3515 - recall: 0.6960 - val_accuracy: 0.8706 - val_loss: 0.3317 - val_recall: 0.6914
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8691 - loss: 0.3489 - recall: 0.6979 - val_accuracy: 0.8640 - val_loss: 0.3395 - val_recall: 0.7060
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8702 - loss: 0.3450 - recall: 0.694

In [ ]:
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    roc_auc_score
)

# ----- 1. Predict Probabilities -----
y_prob = model.predict(x_test)   # shape = (N,1)

# ----- 2. Convert to class labels (0/1) -----
y_pred = (y_prob > 0.25).astype("int32")

# ----- 3. Confusion Matrix -----
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)

# ----- 4. Accuracy -----
acc = accuracy_score(y_test, y_pred)
print("\nAccuracy:", acc)

# ----- 5. Precision / Recall / F1 -----
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# ----- 6. ROC AUC -----
auc = roc_auc_score(y_test, y_prob)
print("\nROC AUC:", auc)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Confusion Matrix:
 [[5238 1608]
 [ 686 2468]]

Accuracy: 0.7706

Classification Report:

              precision    recall  f1-score   support

           0       0.88      0.77      0.82      6846
           1       0.61      0.78      0.68      3154

    accuracy                           0.77     10000
   macro avg       0.74      0.77      0.75     10000
weighted avg       0.80      0.77      0.78     10000


ROC AUC: 0.8785565250994289


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    roc_auc_score
)
import numpy as np

# -------- MODEL WITH KEY PARAMETERS --------
gb = GradientBoostingClassifier(
    loss='log_loss',          # logistic regression loss (for classification)
    learning_rate=0.05,       # shrinkage rate
    n_estimators=400,         # number of boosting stages
    subsample=1.0,            # <1.0 adds randomness (helps generalization)
    criterion='friedman_mse', # default split criterion
    min_samples_split=2,      # min samples to split
    min_samples_leaf=1,       # min samples per leaf
    min_weight_fraction_leaf=0.0,
    max_depth=3,              # depth of individual trees
    min_impurity_decrease=0.0,
    random_state=42,
    max_features=None,        # or "sqrt" / "log2"
    verbose=0,
    validation_fraction=0.1,  # for early stopping
    n_iter_no_change=None,    # set to 10 to enable early stopping
    tol=1e-4,
    ccp_alpha=0.0
)

# -------- TRAIN --------
gb.fit(x_train, y_train)

# -------- PREDICT --------
y_prob = gb.predict_proba(x_test)[:, 1]   # Probabilities
y_pred = (y_prob > 0.5).astype(int)      # Class labels
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print("\nAccuracy:", acc)

# Precision / Recall / F1
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# ROC AUC
auc = roc_auc_score(y_test, y_prob)
print("\nROC AUC:", auc)



Confusion Matrix:
 [[6818   28]
 [1145 2009]]

Accuracy: 0.8827

Classification Report:

              precision    recall  f1-score   support

           0       0.86      1.00      0.92      6846
           1       0.99      0.64      0.77      3154

    accuracy                           0.88     10000
   macro avg       0.92      0.82      0.85     10000
weighted avg       0.90      0.88      0.87     10000


ROC AUC: 0.9095804316023259


KeyError: 'User_ID'